In [ ]:
import cv2
import mediapipe as mp
import pandas as pd
import os
import gc

# Khởi tạo thư viện mediapipe
mpPose = mp.solutions.pose
mpDraw = mp.solutions.drawing_utils

name_points = []
lm_list = []
for i in range(33):
    name_points.append(f"x{i+1}")
    name_points.append(f"y{i+1}")
    name_points.append(f"z{i+1}")
    name_points.append(f"visibility{i+1}")

no_of_frames = 500
def make_landmark_timestep(results):
    print(results.pose_world_landmarks.landmark)
    pose_points = []
    for id, lm in enumerate(results.pose_world_landmarks.landmark):
        pose_points.append(lm.x)
        pose_points.append(lm.y)
        pose_points.append(lm.z)
        pose_points.append(lm.visibility)
    return pose_points

def draw_landmark_on_image(mpDraw, results, img):
    # Vẽ các đường nối
    mpDraw.draw_landmarks(img, results.pose_landmarks, mpPose.POSE_CONNECTIONS)

    # Vẽ các điểm nút
    for id, lm in enumerate(results.pose_landmarks.landmark):
        h, w, c = img.shape
        print(id, lm)
        cx, cy = int(lm.x * w), int(lm.y * h)
        cv2.circle(img, (cx, cy), 10, (0, 0, 255), cv2.FILLED)
    return img
root_folder = "../dataset/Train/"


# Duyệt qua tất cả các video trong thư mục
for subdir, dirs, files in os.walk(root_folder):
    for video_file in files:
        if video_file.endswith(".mp4"):  # Kiểm tra định dạng video
            video_path = os.path.join(subdir, video_file)
            label = os.path.basename(subdir)  # Lấy tên thư mục con làm nhãn

            cap = cv2.VideoCapture(video_path)
            name_video = video_file.split('.')[0]  # Lấy tên video làm nhãn
            print(name_video)

            lm_list = [name_points]
            with mpPose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break
                        # Nhận diện pose
                    frameRGB = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    results = pose.process(frameRGB)

                    if results.pose_world_landmarks:
                        # Ghi nhận thông số khung xương
                        lm = make_landmark_timestep(results)
                        lm_list.append(lm)
                        # Vẽ khung xương lên ảnh
                        frame = draw_landmark_on_image(mpDraw, results, frame)

                    # cv2.imshow("image", frame)
                    # if cv2.waitKey(1) & 0xFF == ord('q'):
                    #     break
            cap.release()
            # Write vào file csv
            df  = pd.DataFrame(lm_list)
            if not os.path.exists(f"landmarks/{label}"):
                os.makedirs(f"landmarks/{label}")
            df.to_csv(f"landmarks/{label}/"+ name_video + ".csv", index=False, header=False)
            # cv2.destroyAllWindows()
            # Dọn bộ nhớ sau mỗi video
            lm_list = []  # Xóa danh sách để tránh đầy bộ nhớ
            gc.collect()  # Giải phóng bộ nhớ không dùng nữa

01502df8-593c-42d0-86ae-7cbd48d8741a
[x: 0.462351203
y: 0.164781779
z: -0.123609379
visibility: 0.999116123
, x: 0.469537914
y: 0.150397182
z: -0.14360632
visibility: 0.999308228
, x: 0.470874608
y: 0.150931418
z: -0.143665865
visibility: 0.999444544
, x: 0.472361028
y: 0.151682138
z: -0.143722504
visibility: 0.99955529
, x: 0.469153523
y: 0.149232388
z: -0.105860487
visibility: 0.99923563
, x: 0.470124513
y: 0.148998111
z: -0.105819009
visibility: 0.999181688
, x: 0.471299499
y: 0.148882389
z: -0.105865173
visibility: 0.999274194
, x: 0.485043436
y: 0.165789902
z: -0.192436352
visibility: 0.999637365
, x: 0.483913124
y: 0.163662761
z: -0.0237635132
visibility: 0.998998106
, x: 0.463948876
y: 0.185880512
z: -0.139331043
visibility: 0.99947685
, x: 0.463507324
y: 0.18362245
z: -0.0904621854
visibility: 0.999083519
, x: 0.49225679
y: 0.277291536
z: -0.278439045
visibility: 0.999980211
, x: 0.495750695
y: 0.273433656
z: 0.0973366499
visibility: 0.999055803
, x: 0.483519822
y: 0.419773877


KeyboardInterrupt: 